### Mount the local drive to save the works

In [ ]:
from google.colab import drive
drive.mount('/content/MyDrive/')

In [ ]:
#!pip install turicreate
import turicreate

In [ ]:
#!7z x '/content/MyDrive/MyDrive/Work/amazon_baby.sframe.zip'

In [14]:
sf = turicreate.SFrame('/content/MyDrive/MyDrive/SFRAMES/amazon_baby.sframe')

In [15]:
sf.head(3)

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0


In [17]:
def remove_punctuation(text):
    import string
    return text.translate(str.maketrans('','',string.punctuation)) #Excellent new method for python 3.0

sf['review_clean'] = sf['review'].apply(remove_punctuation)

In [20]:
sf.head(4)

name,review,rating,review_clean
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0,These flannel wipes areOK but in my opinion not ...
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,it came early and was notdisappointed i love ...
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,Very soft and comfortableand warmer than it ...
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,This is a product wellworth the purchase I ...


In [24]:
#sf = sf.fillna('a', 0)
sf = sf.fillna('review', '')  # fill in N/A's in the review column

In [25]:
#We ignore all rating 3
sf = sf[sf['rating'] != 3]

In [26]:
sf['sentiment'] = sf['rating'].apply(lambda rating : +1 if rating > 3 else -1)

In [27]:
#Train test split
train_data, test_data = sf.random_split(.8, seed=1)

In [29]:
sample_test_data = test_data[10:13]
print(sample_test_data)

+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|   Our Baby Girl Memory Book   | Absolutely love it and all... |  5.0   |
| Wall Decor Removable Decal... | Would not purchase again o... |  2.0   |
| New Style Trailing Cherry ... | Was so excited to get this... |  1.0   |
+-------------------------------+-------------------------------+--------+
+-------------------------------+-----------+
|          review_clean         | sentiment |
+-------------------------------+-----------+
| Absolutely love it and all... |     1     |
| Would not purchase again o... |     -1    |
| Was so excited to get this... |     -1    |
+-------------------------------+-----------+
[3 rows x 5 columns]



In [30]:
sample_test_data[0]['review']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

In [31]:
sample_test_data[1]['review']

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

In [33]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
     # Use this token pattern to keep single-letter words
# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(train_data['review_clean'])
# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(test_data['review_clean'])

In [40]:
#model = turicreate.logistic_classifier.create(data, target='is_expensive', features=['bath', 'bedroom', 'size'])
sentiment_model = turicreate.logistic_classifier.create(train_data, target = 'sentiment', features = ['review_clean'])

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Logistic regression:

--------------------------------------------------------

Number of examples          : 126745

Number of classes           : 2

...

| 2         | 6        | 1.000000  | 1.288138     | 0.999408          | 0.836756            |

| 3         | 8        | 1.000000  | 1.383575     | 0.999408          | 0.836756            |

| 4         | 10       | 1.000000  | 1.479371     | 0.999408          | 0.836756            |

| 9         | 19       | 0.461135  | 1.909423     | 0.999408          | 0.836756            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

In [51]:
#sumval = sentiment_model(sentiment_model.coefficients['value'] > 0).count()
sentiment_model.coefficients['value']
#some_val = sentiment_model.coefficients[2]['value']
#print(sumval)
#print(some_val)

dtype: float
Rows: 126097
[2.2648486006591644, 4.303908046934071, 4.303908046934071, 4.303908046934071, 4.303908046934071, 4.303908046934071, 4.303908046934071, 4.303908046934071, 4.303908046934071, 4.303908046934071, 4.303908046934071, 4.303908046934071, 4.303908046934071, -8.4322088158513, 4.303908046934071, 4.303908046934071, 4.303908046934071, 4.303908046934071, 4.303908046934071, 4.303908046934071, 4.303908046934071, 4.303908046934071, 4.303908046934071, 4.303908046934071, -0.546846696167801, 4.303908046934071, 4.303908046934071, 4.303908046934071, 4.303908046934071, 4.303908046934071, 4.303908046934071, 4.303908046934071, 4.303908046934071, 4.303908046934071, -8.4322088158513, 4.303908046934071, 4.303908046934071, 4.303908046934071, 4.303908046934071, 4.303908046934071, 4.303908046934071, 4.303908046934071, 4.303908046934071, 4.303908046934071, 4.303908046934071, 4.303908046934071, 4.303908046934071, 4.303908046934071, 4.303908046934071, -8.4322088158513, 4.303908046934071, 4.303

In [ ]:
def count_plus():
    return 1

#sf['review_clean'] = sf['review'].apply(remove_punctuation)

In [ ]:
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])
#scores = sentiment_model.decision_function(sample_test_matrix)
#print(scores)